# Build your own GPT

### 1. Choose Training and Model Hyperparameters

In [1]:
import torch, platform

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("CPU:", platform.processor())
print("Platform:", platform.platform())


c:\Users\THE AGAMBISAS\Desktop\Project_IT\GPT-project\gpt_venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:283: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


Torch: 2.10.0+cpu
CUDA available: False
CPU: Intel64 Family 6 Model 78 Stepping 3, GenuineIntel
Platform: Windows-10-10.0.19045-SP0
